# Does resampling experiment help with predicting CbG sentences?

In [1]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import plydata as ply
from sqlalchemy import create_engine

from snorkel.labeling.analysis import LFAnalysis
from snorkeling_helper.generative_model_helper import (
    sample_lfs,
    run_generative_label_function_sampler,
)

warnings.filterwarnings("ignore")

In [2]:
username = "danich1"
password = "snorkel"
dbname = "pubmed_central_db"
database_str = (
    f"postgresql+psycopg2://{username}:{password}@/{dbname}?host=/var/run/postgresql"
)
conn = create_engine(database_str)

## Load the data

In [3]:
label_candidates_dir = Path("../label_candidates/output")
notebook_output_dir = Path("../generative_model_training/output/CbG")

In [4]:
L_abstracts = pd.read_csv(
    str(label_candidates_dir / Path("cg_abstract_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_abstracts.shape)
L_abstracts.head().T

(1292772, 109)


,0,1,2,3,4
LF_HETNET_DRUGBANK,-1,-1,-1,-1,-1
LF_HETNET_DRUGCENTRAL,-1,-1,-1,-1,-1
LF_HETNET_ChEMBL,-1,-1,-1,-1,-1
LF_HETNET_BINDINGDB,-1,-1,-1,-1,-1
LF_HETNET_PDSP_KI,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,0,-1,-1,-1,-1
LF_GG_BICLUSTER_SIGNALING,1,-1,-1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,0,-1,-1,-1,-1


In [5]:
L_dev = pd.read_csv(
    str(label_candidates_dir / Path("cg_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==7")
print(L_dev.shape)
L_dev.head().T

(500, 111)


,0,4,9,10,11
LF_HETNET_DRUGBANK,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_DRUGCENTRAL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_ChEMBL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_BINDINGDB,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_PDSP_KI,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,0.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,0.0,-1.0,-1.0
split,7.0,7.0,7.0,7.0,7.0
document_id,209829.0,178683.0,132454.0,19485.0,358873.0


In [6]:
L_test = pd.read_csv(
    str(label_candidates_dir / Path("cg_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==8")
print(L_test.shape)
L_test.head().T

(500, 111)


,1,2,3,5,6
LF_HETNET_DRUGBANK,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_DRUGCENTRAL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_ChEMBL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_BINDINGDB,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_PDSP_KI,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,-1.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,-1.0,-1.0,-1.0
split,8.0,8.0,8.0,8.0,8.0
document_id,291947.0,321677.0,339647.0,213797.0,30430.0


## Restort Based on the Candidate Abstracts

In [7]:
# Grab the document ids for resampling
sql = """
select cg_candidates.sentence_id, document_id, cg_candidates.candidate_id from sentence
inner join (
  select candidate.candidate_id, compound_gene.sentence_id from compound_gene
  inner join candidate on candidate.candidate_id=compound_gene.candidate_id
  ) as cg_candidates
on sentence.sentence_id = cg_candidates.sentence_id
"""
candidate_doc_df = pd.read_sql(sql, database_str)
candidate_doc_df.head()

,sentence_id,document_id,candidate_id
0,293853012,23111736,17520
1,361497023,24837142,35137
2,395950432,24746171,35735
3,481487831,30853913,42182
4,579412852,21454644,58965


In [8]:
dev_test_ids = (
    L_dev >> ply.select("document_id") >> ply.distinct() >> ply.pull("document_id")
)

filtered_candidate_id = (
    candidate_doc_df
    >> ply.query(f"document_id in {list(dev_test_ids)}")
    >> ply.pull("candidate_id")
)

In [9]:
sorted_train_df = pd.read_csv(
    str(notebook_output_dir / Path("cbg_dataset_mapper.tsv")), sep="\t"
)
sorted_train_df.head()

,document_id,dataset
0,23111736,train
1,24837142,train
2,24746171,train
3,30853913,tune
4,21454644,train


In [10]:
trained_documents = (
    sorted_train_df
    >> ply.inner_join(candidate_doc_df, on="document_id")
    >> ply.query("dataset=='train'")
    >> ply.pull("candidate_id")
)

In [11]:
filtered_L_abstracts = L_abstracts >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_abstracts.shape)
filtered_L_abstracts.head()

(904644, 109)


,LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HETNET_ChEMBL,LF_HETNET_BINDINGDB,LF_HETNET_PDSP_KI,LF_HETNET_US_PATENT,LF_HETNET_PUBCHEM,LF_HETNET_CG_ABSENT,LF_CG_CHECK_GENE_TAG,LF_DG_IS_BIOMARKER,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,191
7,-1,-1,-1,-1,-1,-1,-1,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,449
8,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,450
10,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,897
11,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,898


## Construct the Grid Search

In [12]:
# Global Grid
epochs_grid = [500]
l2_param_grid = np.linspace(0.01, 5, num=5)
lr_grid = [1e-2]
grid = list(
    zip(epochs_grid * len(l2_param_grid), l2_param_grid, lr_grid * len(l2_param_grid))
)

# Abstracts

In [13]:
analysis_module = LFAnalysis(
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
)

abstract_lf_summary = analysis_module.lf_summary()
abstract_lf_summary.index = (
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
).columns.tolist()

abstract_lf_summary

,Polarity,Coverage,Overlaps,Conflicts
LF_HETNET_DRUGBANK,[1],0.080205,0.080205,0.080205
LF_HETNET_DRUGCENTRAL,[1],0.026783,0.026783,0.026783
LF_HETNET_ChEMBL,[1],0.030188,0.030188,0.030188
LF_HETNET_BINDINGDB,[1],0.004332,0.004332,0.004332
LF_HETNET_PDSP_KI,[1],0.003062,0.003062,0.003062
...,...,...,...,...
LF_GG_BICLUSTER_AFFECTS_EXPRESSION,[1],0.022110,0.022110,0.022110
LF_GG_BICLUSTER_INCREASES_EXPRESSION,[0],0.039291,0.039291,0.039291
LF_GG_BICLUSTER_SIGNALING,[1],0.045772,0.045772,0.045772
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,[0],0.027517,0.027517,0.027517


# Set up fields for resampling

In [14]:
lf_columns_base = list(L_abstracts.columns[0:9])
candidate_id_field = list(L_abstracts.columns[-1:])
dev_column_base = ["split", "curated_cbg", "document_id"]
data_columns = []

# Abstracts - All Label Functions

In [15]:
cbg_start = 9
cbg_end = 108

# Spaced out number of sampels including total
size_of_samples = [1, 33, 65, 97, cbg_end - cbg_start]
number_of_samples = 50
cbg_lf_range = range(cbg_start, cbg_end)

In [16]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [17]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="ALL",
    data_source="abstract",
)

100%|██████████| 50/50 [24:48<00:00, 29.77s/it]


# Write Performance to File

In [18]:
performance_df = pd.DataFrame.from_records(data_columns)
performance_df

,lf_num,auroc,aupr,bce_loss,sampled_lf_name,label_source,data_source,model,epochs,l2_param,lr_param
0,1,0.634260,0.166257,0.313810,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,tune,500,1.2575,0.01
1,1,0.848545,0.333780,0.203579,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,test,500,1.2575,0.01
2,1,0.572559,0.155960,2.530169,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,tune,500,0.0100,0.01
3,1,0.719066,0.272369,2.821523,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,test,500,0.0100,0.01
4,1,0.571303,0.161867,0.339584,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,tune,500,1.2575,0.01
...,...,...,...,...,...,...,...,...,...,...,...
495,99,0.649288,0.134816,1.794591,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,test,500,5.0000,0.01
496,99,0.541416,0.085957,2.795459,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,tune,500,5.0000,0.01
497,99,0.649288,0.134816,1.794591,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,test,500,5.0000,0.01
498,99,0.541416,0.085957,2.795459,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",ALL,abstract,tune,500,5.0000,0.01


In [19]:
(
    performance_df
    >> ply.call(
        "to_csv",
        str(Path("output") / Path("ALL_CbG_performance.tsv")),
        index=False,
        sep="\t",
    )
)